In [ ]:
import pandas as pd
import fastf1 as f1 


# schedule for 2022 season 
schedule22 = f1.get_event_schedule(2022, include_testing=False)
schedule_2022 = schedule22[['RoundNumber', 'Location', 'EventName']]

# schedule for 2023 season 
schedule23 = f1.get_event_schedule(2023, include_testing=False)
schedule_2023 = schedule23[['RoundNumber', 'Location', 'EventName']]


# This is what I am expecting the result to contain (Doesn't have to be in the same order) 
""" drivers_2022_list = ["Zhou Guanyu", "Valtteri Bottas", "Pierre Gasly", "Yuki Tsunoda","Fernando Alonso", "Esteban Ocon","Sesbastian Vettel","Lance Stroll","Charles Leclerc","Carlos Sainz Jr.","Kevin Magnussen","Mick Schumacher","Daniel Ricciardo", "Lando Norris","Lewis Hamilton","George Russell"," Max Verstappen"," Sergio Perez", "Nicholas Latifi", "Alexander Albon"] """ 


# I am choosing the Australian GP randomly because it contains all the main drivers in their main team.  
F1_2022_AUS = f1.get_session(2022, "Australian Grand Prix", "R" )
F1_2022_AUS.load(telemetry=True, laps=True)
drivers_2022_list = F1_2022_AUS.results['FullName'].tolist()

F1_2023_AUS = f1.get_session(2023, 'Australian Grand Prix', 'R')
F1_2023_AUS.load()
drivers_2023_list = F1_2023_AUS.results['FullName'].tolist()
drivers_2023_list.append('Daniel Ricciardo')


core           INFO 	Loading data for Australian Grand Prix - Race [v3.5.3]
req            INFO 	Using cached data for session_info
req            INFO 	Using cached data for driver_info
req            INFO 	Using cached data for session_status_data
req            INFO 	Using cached data for lap_count
req            INFO 	Using cached data for track_status_data
req            INFO 	Using cached data for _extended_timing_data
req            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...
req            INFO 	Using cached data for car_data
req            INFO 	Using cached data for position_data
req            INFO 	Using cached data for weather_data
req            INFO 	Using cached data for race_control_messages
core        WARNING 	Driver 16 completed the race distance 00:00.140000 before the recorded end of the session.
core           INFO 	Finished loading data for 20 drivers: ['16', '11', '63', '44', '4', '3', '31', '77', '10', '23', '24',

Charles Leclerc
Sergio Perez
George Russell
Lewis Hamilton
Lando Norris
Daniel Ricciardo
Esteban Ocon
Valtteri Bottas
Pierre Gasly
Alexander Albon
Guanyu Zhou
Lance Stroll
Mick Schumacher
Kevin Magnussen
Yuki Tsunoda
Nicholas Latifi
Fernando Alonso
Max Verstappen
Sebastian Vettel
Carlos Sainz


req            INFO 	Using cached data for car_data
req            INFO 	Using cached data for position_data
req            INFO 	Using cached data for weather_data
req            INFO 	Using cached data for race_control_messages
core           INFO 	Finished loading data for 20 drivers: ['1', '44', '14', '18', '11', '4', '27', '81', '24', '22', '77', '55', '10', '31', '21', '2', '20', '63', '23', '16']


Max Verstappen
Lewis Hamilton
Fernando Alonso
Lance Stroll
Sergio Perez
Lando Norris
Nico Hulkenberg
Oscar Piastri
Guanyu Zhou
Yuki Tsunoda
Valtteri Bottas
Carlos Sainz
Pierre Gasly
Esteban Ocon
Nyck De Vries
Logan Sargeant
Kevin Magnussen
George Russell
Alexander Albon
Charles Leclerc
Daniel Ricciardo
